In [1]:
import os
os.environ['HF_DATASETS_CACHE'] = '/workspace/.cache/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/workspace/.cache/huggingface'

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import PeftModel, PeftConfig
import huggingface_hub

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
huggingface_hub.login('<Your Access Token>')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# base_model = "beomi/Llama-3-Open-Ko-8B"
base_model = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_model_id = './model_ckpt/checkpoint-123930'
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

merged_model = model.merge_and_unload()

config = model.config
# _name_or_path 변경
config._name_or_path = 'yainage90/Llama3-open-Ko-3-8B-Law-Chat-Full-Weights'
tokenizer.save_pretrained('./Llama3-open-Ko-3-8B-Law-Chat-Full-Weights')
merged_model.save_pretrained('./Llama3-open-Ko-3-8B-Law-Chat-Full-Weights')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
api = huggingface_hub.HfApi()

api.upload_folder(
    folder_path='./Llama3-open-Ko-3-8B-Law-Chat-Full-Weights',
    repo_id='yainage90/Llama3-open-Ko-3-8B-Law-Chat-Full-Weights',
    repo_type="model",
)